# 🛰️ Classificação de Uso do Solo com Imagens Sentinel-2
Este notebook demonstra, de forma interativa, o pipeline completo de classificação do uso do solo com base em imagens Sentinel-2.

## 🔧 Configuração do Ambiente
Importação de bibliotecas e módulos locais.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import psutil
import gc

# Verificar uso de memória
def check_memory():
    """Verifica o uso atual de memória"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    print(f"🧠 Uso de memória: {mem_info.rss / 1024 / 1024:.1f} MB")
    print(f"💾 Memória disponível: {psutil.virtual_memory().available / 1024 / 1024:.1f} MB")

print("🔧 Configurando ambiente...")
check_memory()

# Importar módulos com tratamento de erros
try:
    from data_loader import SentinelDataLoader
    print("✅ data_loader importado com sucesso")
except Exception as e:
    print(f"❌ Erro ao importar data_loader: {e}")
    sys.exit(1)

try:
    from ml_models import MLModels
    print("✅ ml_models importado com sucesso")
except Exception as e:
    print(f"❌ Erro ao importar ml_models: {e}")

try:
    from hyperparameter_optimization import HyperparameterOptimizer
    print("✅ hyperparameter_optimization importado com sucesso")
except Exception as e:
    print(f"❌ Erro ao importar hyperparameter_optimization: {e}")

print("🚀 Importações concluídas!")


## 📥 Carregamento e Pré-processamento de Dados

In [ ]:

print("📥 Iniciando carregamento de dados...")
check_memory()

try:
    loader = SentinelDataLoader(data_dir='ImagensSentinel-2')
    print("✅ SentinelDataLoader inicializado")
except Exception as e:
    print(f"❌ Erro ao inicializar loader: {e}")
    raise

try:
    print("🔄 Carregando bandas B02, B03, B04...")
    loader.bands = {}
    for band in ["B02", "B03", "B04"]:
        loader.bands[band] = loader.load_band(band)
        check_memory()

    gc.collect()

    print("🔄 Carregando SCL...")
    loader.load_scl()
    check_memory()

except MemoryError:
    print("❌ Erro de memória! Tentando limpeza...")
    gc.collect()
    check_memory()
    raise
except Exception as e:
    print(f"❌ Erro no carregamento: {e}")
    raise

N_SAMPLES = 50
print(f"🔄 Amostrando {N_SAMPLES} dados...")

try:
    features, targets = loader.sample_data(n_samples=N_SAMPLES)
    print(f"✅ Dados amostrados: {features.shape[0]} amostras, {features.shape[1]} features")

    X_train, X_val, X_test, y_train, y_val, y_test = loader.split_data(features, targets)

    print("📊 Divisão dos dados:")
    print(f"   Treino: {len(X_train)} amostras")
    print(f"   Validação: {len(X_val)} amostras")
    print(f"   Teste: {len(X_test)} amostras")

    train_dist = loader.get_class_distribution(y_train)
    print(f"📈 Distribuição das classes (treino):")
    for classe, count in train_dist.items():
        print(f"   {classe}: {count} ({(count / len(y_train)) * 100:.1f}%)")

    check_memory()

except Exception as e:
    print(f"❌ Erro na amostragem: {e}")
    raise

print("✅ Carregamento concluído com sucesso!")


## 📊 Visualização das Bandas RGB e Máscara SCL

In [ ]:

print("📊 Iniciando visualizações...")

try:
    print("🔄 Criando composição RGB...")
    loader.visualize_rgb()
    print("✅ RGB visualizado")
except Exception as e:
    print(f"⚠️ Erro na visualização RGB: {e}")

gc.collect()
check_memory()

try:
    print("🔄 Visualizando SCL...")
    loader.visualize_scl()
    print("✅ SCL visualizado")
except Exception as e:
    print(f"⚠️ Erro na visualização SCL: {e}")

gc.collect()
check_memory()
print("✅ Visualizações concluídas!")


## 🤖 Treinamento dos Modelos de Machine Learning

In [ ]:

ml_models = MLModels()
ml_models.add_random_forest()
ml_models.add_svm()
ml_models.add_neural_network()
ml_models.train_all_models(X_train, y_train, X_val, y_val)


## 🧪 Avaliação dos Modelos no Conjunto de Teste

In [ ]:

for model_name in ml_models.models.keys():
    ml_models.evaluate_model(model_name, X_test, y_test)


## 🔬 Otimização de Hiperparâmetros (opcional e demorado)

In [ ]:

# ⚠️ Cuidado: esta etapa é demorada
# optimizer = HyperparameterOptimizer(cv=3)
# optimized_models = optimizer.optimize_all_models(X_train, y_train)
# optimizer.compare_optimization_results()


## ✅ Conclusão
Este notebook mostra o processo completo de leitura, visualização, treinamento e avaliação do modelo.